In [ ]:
!pip install rasterio
!pip install fiona
!pip install pysatproc

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd '/content/drive/MyDrive/Basurales/AMBA/labels/'

/content/drive/MyDrive/Basurales/AMBA/labels


In [ ]:
# Este paso requiere Orfeo ToolBox! - PENDIENTE: Resolver instalación de OTB en Google Colab
# Sirve para las imágenes descargadas via GEE, que traen las bandas no visibles.
# Pero no para las de Copernicus, que solo traen RGB

import glob
import os
from tqdm import tqdm

PATHS = [#'/tmp/01 - IMAGENES SENTINEL/*.tif',
         '/tmp/ALGORITMO/rasters/*.tif',
         ]

#Computar NDSW index
# Generamos la banda Normalized Difference ShortWave (NDSW) 
# como la diferencia normalizada entre las dos bandas del infrarrojo de onda corta, 
# es decir NDSW = (SWIR12-SWIR11)/(SWIR12 + SWIR11 ) . 
# Esta diferencia de bandas mostró ser sensible a la emisión de metano en estudios previos. 
# Dicho componente puede ser un indicador de la presencia de un basural con residuos biológicos.


# para cada imagen en los grupos de entrenamiento generamos la banda NDSW
for path in PATHS:
   
    images = glob.glob(path) #ruta a cada imagen 
    
    for image in tqdm(images):
        
        #skip resulting files 
        if "_diffSW" in image: continue
        if "_ori_tmp" in image: continue
        if "rgb-nir-sw-diffSW"  in image: continue
            
        fname, ext = os.path.splitext(image)
        
        #--------------------------------
        #  Generacion de la banda NDSW  
        #--------------------------------
        
        ndsw_file = str(fname+'_diffSW.tif')
        #usamos gdal_calc para calcular la nueva banda usando la banda 5 y 6 de cada imagen
        !gdal_calc.py -A $image --A_band=5 -B $image --B_band=6 --calc="(A-B)/(A+B)" --outfile=$ndsw_file 
        
        #------------------------------------------
        #  Generacion de la bandas RGB - NIR -SWIR  
        #------------------------------------------
        
        img_5bands = str(fname+'_ori_tmp.tif')
        #usamos gdal_translate quedarnos con las 5 primeras bandas de cada imagen
        !gdal_translate -b 1 -b 2 -b 3 -b 4 -b 5 $image $img_5bands

        #----------------------------------------------
        #  Generacion de imagen RGB - NIR -SWIR -NDSW 
        #----------------------------------------------        
        
        rgb_nir_swir_diffSW_fname = str(fname+'_rgb-nir-sw-diffSW.tif')

        #usamos otbcli_ConcatenateImages para concatenar las imagenes generadas en el paso anterior quedarnos con las 5 primeras bandas de cada imagen
        !bash -c "source /opt/OTB/otbenv.profile; otbcli_ConcatenateImages -il  $img_5bands $ndsw_file -out $rgb_nir_swir_diffSW_fname"


0it [00:00, ?it/s]


In [ ]:
import pathlib

annotations_dir = '/content/drive/MyDrive/Basurales/AMBA/ENTREGAS_MASCARAS/DICIEMBRE_2022/ANOTACIONES/'
output_dir = '/content/drive/MyDrive/Basurales/AMBA/labels/CHIPS/'
chip_size = 160
step_size = 48


In [ ]:
def process_folder(location, output_dir, chip_size, step_size):
    image = location + "/*_RGB*.tif"
    labels = location + "/*.gpkg"

    !satproc_extract_chips \
        $image \
        -o  $output_dir \
        --size $chip_size \
        --step-size $step_size \
        --aoi $labels \
        --labels $labels \
        --label-property 'clase' \
        --classes 'basural'

In [ ]:
for location in map(lambda x: x.as_posix(), pathlib.Path(annotations_dir).iterdir()):
    print("Extracting chips from " + location)
    process_folder(location, output_dir, chip_size, step_size)

Extracting chips from /content/drive/MyDrive/Basurales/AMBA/ENTREGAS_MASCARAS/DICIEMBRE_2022/ANOTACIONES/2021-12
Rasters:   0% 0/1 [00:00<?, ?it/s]
DIC-2021-AMBA-SENTINEL_RGB_BIS.tif windows:   0% 0/5586 [00:00<?, ?it/s]
DIC-2021-AMBA-SENTINEL_RGB_BIS.tif windows:   4% 206/5586 [00:00<00:02, 2054.98it/s]
DIC-2021-AMBA-SENTINEL_RGB_BIS.tif windows:   7% 412/5586 [00:00<00:02, 2006.73it/s]
DIC-2021-AMBA-SENTINEL_RGB_BIS.tif windows:  11% 613/5586 [00:00<00:02, 1844.46it/s]
DIC-2021-AMBA-SENTINEL_RGB_BIS.tif windows:  15% 824/5586 [00:00<00:02, 1912.62it/s]
DIC-2021-AMBA-SENTINEL_RGB_BIS.tif windows:  18% 1017/5586 [00:00<00:02, 1809.62it/s]
DIC-2021-AMBA-SENTINEL_RGB_BIS.tif windows:  21% 1200/5586 [00:00<00:02, 1630.77it/s]
DIC-2021-AMBA-SENTINEL_RGB_BIS.tif windows:  24% 1366/5586 [00:00<00:02, 1484.70it/s]
DIC-2021-AMBA-SENTINEL_RGB_BIS.tif windows:  28% 1539/5586 [00:00<00:02, 1549.75it/s]
DIC-2021-AMBA-SENTINEL_RGB_BIS.tif windows:  30% 1699/5586 [00:01<00:02, 1563.38it/s]
DIC-2021-